## 업비트 API 실습

In [ ]:
import os
import jwt  # PyJWT
import uuid
import hashlib
from urllib.parse import urlencode
import requests

### 환경변수 호출

In [ ]:
from dotenv import load_dotenv

load_dotenv()

ACCESS_KEY = os.environ["UPBIT_OPEN_API_ACCESS_KEY"]
SECRET_KEY = os.environ["UPBIT_OPEN_API_SECRET_KEY"]
SERVER_URL = os.environ["UPBIT_OPEN_API_SERVER_URL"]
SERVER_URL

### 토큰 만들기

In [ ]:
def create_token(self, query=None):
    """API에 사용될 토큰을 생성"""
    if query is None:
        print("Start create token without query")
        payload = {
            "access_key": ACCESS_KEY,
            "nonce": str(uuid.uuid4()),
        }

        jwt_token = jwt.encode(payload, SECRET_KEY)
        authorization_token = "Bearer {}".format(jwt_token)
        return authorization_token

    # query는 dict 타입입니다.
    print("Start create token without query")
    print(query)
    m = hashlib.sha512()
    m.update(urlencode(query).encode())
    print("url encoded_query")
    print(urlencode(query).encode())
    query_hash = m.hexdigest()

    print("hash of encoded_query")
    print(query_hash)

    payload = {
        "access_key": ACCESS_KEY,
        "nonce": str(uuid.uuid4()),
        "query_hash": query_hash,
        "query_hash_alg": "SHA512",
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorization_token = "Bearer {}".format(jwt_token)
    return authorization_token

### 최근 거래 내역 조회

In [ ]:
def query_latest_trade(market):
    """최근 거래 내역 조회
    response:
        trade_date_utc: 체결 일자(UTC 기준), String
        trade_time_utc: 체결 시각(UTC 기준), String
        timestamp: 체결 타임스탬프, Long
        trade_price: 체결 가격, Double
        trade_volume: 체결량, Double
        prev_closing_price: 전일 종가, Double
        change_price: 변화량, Double
        ask_bid: 매도/매수, String
        sequential_id: 체결 번호(Unique), Long
    """
    querystring = {
        "market": market,
        "count":"1"
    }

    response = requests.request("GET", SERVER_URL + "/v1/trades/ticks", params=querystring)

    print(response.json())
    return response.json()
query_latest_trade('KRW-BTC')